# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
raw_df = pd.read_csv('../output_data/WeatherPy_output/raw_weather.csv')
print(len(raw_df))
raw_df.head()

619


,id,main,description,icon,base,visibility,dt,timezone,name,cod,...,filler_18,filler_19,filler_20,filler_21,filler_22,filler_23,filler_24,filler_25,filler_26,rain.1h
0,802.0,Clouds,scattered clouds,03d,stations,10000.0,1.608738e+09,-10800.0,Belmonte,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,803.0,Clouds,broken clouds,04n,stations,10000.0,1.608738e+09,-36000.0,Hilo,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,803.0,Clouds,broken clouds,04n,stations,10000.0,1.608738e+09,3600.0,Honningsvåg,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,802.0,Clouds,scattered clouds,03d,stations,10000.0,1.608738e+09,-3600.0,Lagoa,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,801.0,Clouds,few clouds,02n,stations,9000.0,1.608738e+09,28800.0,George Town,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [34]:
import gmaps
gmaps.configure(api_key=g_key)

In [43]:
coords = raw_df[['coord.lat', 'coord.lon']].fillna(0)
humidity = raw_df['main.humidity']

fig = gmaps.figure(map_type='HYBRID')

heatmap_layer = gmaps.heatmap_layer(coords, weights=humidity, dissipating=False, max_intensity=10, point_radius=1)

fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
ideal_temp_mask = (raw_df['main.temp_max'] < 80) & (raw_df['main.temp_max'] > 70)
ideal_wind_mask = raw_df['wind.speed'] < 10
ideal_cloud_mask = raw_df['clouds.all'] == 0 

ideal_spots_df = raw_df[ideal_temp_mask & ideal_wind_mask & ideal_cloud_mask]
ideal_spots_df.index = ideal_spots_df['name']
ideal_spots_df.head()

,id,main,description,icon,base,visibility,dt,timezone,name,cod,...,filler_18,filler_19,filler_20,filler_21,filler_22,filler_23,filler_24,filler_25,filler_26,rain.1h
name,,,,,,,,,,,,,,,,,,,,,
Tessalit,800.0,Clear,clear sky,01d,stations,10000.0,1.608738e+09,0.0,Tessalit,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bredasdorp,800.0,Clear,clear sky,01d,stations,10000.0,1.608738e+09,7200.0,Bredasdorp,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
São Pedro do Sul,800.0,Clear,clear sky,01d,stations,10000.0,1.608738e+09,-10800.0,São Pedro do Sul,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Yulara,800.0,Clear,clear sky,01n,stations,10000.0,1.608738e+09,34200.0,Yulara,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Taoudenni,800.0,Clear,clear sky,01d,stations,10000.0,1.608738e+09,0.0,Taoudenni,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
ideal_coords = ideal_spots_df[['coord.lat','coord.lon']]
string_coords = []
responses = []

for vacation_spot in range(len(ideal_coords)):
    str_coord = f'{str(ideal_coords.iloc[vacation_spot, 0])}, {str(ideal_coords.iloc[vacation_spot,1])}'
    string_coords.append(str_coord)

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
target_search = "Hotel"
target_radius = 5000
target_type = "hotel"

for vacation_spot in string_coords: 

    params = {
        "location": vacation_spot,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    response = requests.get(base_url, params=params)
    responses.append(response)

In [141]:
from pprint import pprint
clean_responses = [resp for resp in responses if resp.json()['status'] != 'ZERO_RESULTS']
pprint(clean_responses[0].json()['results'][0])

{'business_status': 'OPERATIONAL',
 'geometry': {'location': {'lat': -34.5326546, 'lng': 20.0399175},
              'viewport': {'northeast': {'lat': -34.53134607010728,
                                         'lng': 20.04134342989272},
                           'southwest': {'lat': -34.53404572989272,
                                         'lng': 20.03864377010727}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/cafe-71.png',
 'name': 'Bredasdorp Square',
 'opening_hours': {'open_now': False},
 'photos': [{'height': 2976,
             'html_attributions': ['<a '
                                   'href="https://maps.google.com/maps/contrib/109411988681379351778">Coenraad '
                                   'Grunschloss</a>'],
             'photo_reference': 'ATtYBwJRzV_5V569-hkeCUSnvLMcMy9AoPk7HS7qlqPb4qhrm3LOigbOdH70eViu2nHlsR-O38gmWVrot3CwDD-HRY4gwiVVXITJVacvT1FeQkBjsoAB90c7ILZ26YMtf6PyNwRUNhyokBIe798jUKtlpGsHyU2o8GJZBw9JDlhSZnRrXwX-',
             'wid

In [138]:
hotel_df = {}
lat = {}
lng = {}

i = 0
for response in clean_responses:
    city_name = f'{ideal_spots_df.iloc[i,8]}'
    hotel_json = response.json()
    hotel_df[city_name] = hotel_json['results'][0]['name']
    lat[city_name] = [hotel_json['results'][0]['geometry']['location']['lat']]
    lng[city_name] = [hotel_json['results'][0]['geometry']['location']['lng']]
    i += 1

cities_with_hotel = [city for city in hotel_df]

ideal_spots_df.loc['Hotel Name'] = np.nan

for city in cities_with_hotel:
    ideal_spots_df.loc[city, 'Hotel Name'] = hotel_df[f'{city}']
    
hotel_df = ideal_spots_df['Hotel Name']

hotel_df = pd.DataFrame(hotel_df)

ideal_spots_df.head()

,id,main,description,icon,base,visibility,dt,timezone,City,cod,...,filler_19,filler_20,filler_21,filler_22,filler_23,filler_24,filler_25,filler_26,rain.1h,Hotel Name
name,,,,,,,,,,,,,,,,,,,,,
Tessalit,800.0,Clear,clear sky,01d,stations,10000.0,1.608738e+09,0.0,Tessalit,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bredasdorp Square
Bredasdorp,800.0,Clear,clear sky,01d,stations,10000.0,1.608738e+09,7200.0,Bredasdorp,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sítio Hotel e Eventos
São Pedro do Sul,800.0,Clear,clear sky,01d,stations,10000.0,1.608738e+09,-10800.0,São Pedro do Sul,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sails in the Desert
Yulara,800.0,Clear,clear sky,01n,stations,10000.0,1.608738e+09,34200.0,Yulara,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hospitality Carnarvon
Taoudenni,800.0,Clear,clear sky,01d,stations,10000.0,1.608738e+09,0.0,Taoudenni,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Free Zone Hotel Nouadhibou


In [142]:
hotel_df = hotel_df.dropna()

In [149]:
hotel_df['City'] = hotel_df.index

hotel_df['Lat'] = pd.DataFrame(lat).transpose() 
hotel_df['Lng'] = pd.DataFrame(lng).transpose()

# Get proper country --> markers mislabeling hotels, proper coords

# hotel_df['Country'] = ideal_spots_df['sys.country']

# hotel_df

KeyError: 21

In [133]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [134]:
hotel_names = hotel_df['Hotel Name'].values.tolist()
hotel_names

['Bredasdorp Square',
 'Sítio Hotel e Eventos',
 'Sails in the Desert',
 'Hospitality Carnarvon',
 'Free Zone Hotel Nouadhibou',
 'Hotel Castelo',
 'HÔTEL SAHRAI',
 'Holiday Inn Express Iquique',
 'Hotel Mar Andino',
 'Turis Firper Hotel',
 'Melrose',
 'Juddi Hotel',
 'Hotel Dom Rafael - Premium',
 'Dar One',
 'Hotel Emblemático 4 Esquinas',
 'Hotel Piedra Verde Rancagua Machalí']

In [135]:
# Add marker layer ontop of heat map

hotel_names = hotel_df['Hotel Name'].values.tolist()

marker_layer = gmaps.marker_layer(locations, hotel_names, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))